In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.decomposition import PCA

In [2]:
root = '../sumegh/GabData/'
! ls -lh '../sumegh/GabData/'

total 8.3G
-rw-r--r-- 1 btp-souvic btp-souvic  140M Aug 17 16:57 dict_user_postRepost.pklgz
-rw-r--r-- 1 btp-souvic btp-souvic  7.0G Aug 17 16:59 Final_Posts.pkl
-rw-r--r-- 1 btp-souvic btp-souvic  214M Aug 17 17:16 gabEdges1_5degree.pklgz
-rw-r--r-- 1 btp-souvic btp-souvic  6.1K Aug 17 17:17 haters.json
-rw-r--r-- 1 btp-souvic btp-souvic  5.2K Aug 17 17:18 nonhaters.json
-rw-r--r-- 1 btp-souvic btp-souvic 1023M Aug 17 17:00 user_follower_following.json


In [10]:
posts = pickle.load(open(root+"Final_posts.pkl", "rb"))

In [4]:
vectors = []
for i, post in enumerate(tqdm(posts)):
    t = posts[post]
    vector = []
    vec = [0] * 15
    if t['category_value'] is not None:
        vec[t['category_value'] - 1] += 1
    vector.extend(vec)
    vec = [0] * 15
    if t['topic_category'] is not None:
        vec[t['topic_category'] - 1] += 1
    vector.extend(vec)
    vectors.append(vector)
vectors = np.array(vectors)
vectors = PCA(n_components=6).fit_transform(vectors)

100%|██████████| 21207961/21207961 [01:27<00:00, 241061.04it/s]


In [25]:
extracted_posts = []
col_names = ['id','ancestor_id','user_id','feature']
for i, post in enumerate(tqdm(posts)):
    t = posts[post]
    vector = []
    vector.append(t['score'])
    vector.append(t['like_count'])
    vector.append(t['dislike_count'])
    vector.append(t['repost_count'])
    vector.extend([round(x,4) for x in vectors[i]])
    t2 = dict()
    t2[col_names[0]] = post
    t2[col_names[1]] = t['conversation_parent_id']
    t2[col_names[2]] = t['userid']
    t2[col_names[3]] = vector
    extracted_posts.append(t2)
df = pd.DataFrame(extracted_posts)

100%|██████████| 21207961/21207961 [10:44<00:00, 32889.53it/s] 


In [28]:
df.to_csv('extracted.csv', index=None)